<a href="https://colab.research.google.com/github/MMv110/LaboTeoria/blob/main/SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from pprint import pprint
import re
from bs4 import BeautifulSoup
import numpy as np

#Leer el corpus
data = open("/content/drive/MyDrive/Lingüística computacional/SMS_Spam_Corpus_big.txt", encoding="ISO-8859-1")

# Separa los datos en X e y
(y, X) = zip(*[(1 if nltk.word_tokenize(text.lower())[-1] == "spam" else 0, nltk.word_tokenize(text.lower())[:-2]) for text in data])

print(X[0:2])
print(y[0:2])


(['urgent', '!', 'call', '09061749602', 'from', 'landline', '.', 'your', 'complimentary', '4', '*', 'tenerife', 'holiday', 'or', '£10,000', 'cash', 'await', 'collection', 'sae', 't', '&', 'cs', 'box', '528', 'hp20', '1yf', '150ppm', '18+'], ['+449071512431', 'urgent', '!', 'this', 'is', 'the', '2nd', 'attempt', 'to', 'contact', 'u', '!', 'u', 'have', 'won', '£1250', 'call', '09071512433', 'b4', '050703', 't', '&', 'csbcm4235wc1n3xx', '.', 'callcost', '150ppm', 'mobilesvary', '.', 'max£7', '.', '50'])
(1, 1)


In [2]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = np.transpose(np.expand_dims(y_train,1))
y_test = np.transpose(np.expand_dims(y_test,1))

In [3]:
# Crea un conjunto vacío para almacenar el vocabulario
vocabulary = set()

# Itera sobre los arreglos y agrega cada palabra al conjunto del vocabulario
for array in X:
    for word in array:
        vocabulary.add(word)

vocabulary = sorted(vocabulary)

# Imprime el vocabulario
print(vocabulary)

['!', '#', '$', '%', '&', "'", "''", "'help", "'ll", "'m", "'re", "'s", "'ve", '(', ')', '*', '+', '+123', '+447797706009', '+449071512431', '+cheer', '+get', '+std', '+£400', ',', '-', '-call', '-message', '-no', '-sub', '.', '..', '...', '....', '.....', '.terry', '//careers', '//www.urawinner.com', '/colour', '0', '008704050406', '0121', '01223585334', '0125698789', '02', '02/06/03', '02/09/03', '02072069400.', '02073162414', '021', '050703', '0578', '07090201529', '07123456789', '07732584351', '07734396839', '07742676969', '0776xxxxxxx', '07786200117', '077xxx', '07801543489', '07821230901', '0789xxxxxxx', '07946746291/07880867867', '07xxxxxxxxx', '0800', '08000407165', '08000839402', '08000930705', '08000930705.', '08000938767', '08001950382', '08002986030', '08002986906', '08006344447', '0808', '08081263000', '08081560665', '0825', '0844', '0845', '08452810073', '08452810075over18', '0870', '08701237397', '08701417012', '08701417012150p', '0870141701216+', '0870241182716', '08702

In [4]:
import collections

# Crea la matriz tdm
def term_document_matrix(documents, voc):
  # Matriz de términos documentos
  matriz = np.zeros((len(voc), len(documents)))

  # Contar las ocurrencias de cada término en cada documento
  for i, documento in enumerate(documents):
    contador = collections.Counter(documento)
    for término, ocurrencia in contador.items():
      matriz[voc.index(término)][i] = ocurrencia

  # Dividir la ocurrencia de cada término en cada documento por la suma de todas las ocurrencias
  for i in range(len(matriz)):
    suma = np.sum(matriz[i])
    if suma != 0:
        matriz[i] /= suma

  return matriz

tdm = term_document_matrix(x_train, vocabulary)

In [5]:
import numpy as np
from scipy.special import expit

#Asignamos hiper-parámetros
epochs = 10000
alfa = 10

# Inicializamos w
w = np.zeros((len(tdm),1))

# Bucle de entrenamiento
for epoch in range(epochs):

  # Calculamos la predicción
  y_pred =  expit(-np.dot(w.T, tdm))

  # Calculamos el error
  val_1 = y_train
  val_2 = np.log1p(y_pred)
  val_3 = 1-y_train
  val_4 = np.log1p(1-y_pred)
  val_5 = np.multiply(- val_1, val_2)
  val_6 = np.multiply(val_3, val_4)
  val_7 = np.subtract(val_5, val_6)
  error = np.mean(val_7)

  if epoch % (epochs/10) == 0:
      print(error)

  # Actualizamos los pesos
  w -= (alfa * np.dot(np.subtract(y_pred, y_train), tdm.T)).T


-0.40546510810816455
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874
-0.00196358974662874


In [6]:

tdm_f = term_document_matrix(x_test, vocabulary)

# Calculamos la predicción
y_pred_f =  expit(-np.dot(w.T, tdm_f))
y_pred_f = np.transpose(y_pred_f)

pred = np.zeros(len(y_pred_f))
for i, y_f in enumerate(y_pred_f):
  if y_f <= 0.5:
     pred[i]= 1
  else:
    pred[i] = 0

In [7]:
# Matriz de confusión
matriz_confusion = np.zeros((2, 2))

# Contar las coincidencias
for prediccion, etiqueta_real in zip(pred, y_test[0]):
  matriz_confusion[int(prediccion)][int(etiqueta_real)] += 1

# Imprimir la matriz de confusión
print(matriz_confusion)

# Calcula las métricas
precision = matriz_confusion[0, 0] / (matriz_confusion[0, 0] + matriz_confusion[1, 0])
specificity = matriz_confusion[1, 1] / (matriz_confusion[1, 1] + matriz_confusion[0, 1])
recall = matriz_confusion[0, 0] / (matriz_confusion[0, 0] + matriz_confusion[0, 1])
fallout = matriz_confusion[1, 0] / (matriz_confusion[1, 0] + matriz_confusion[1, 1])
accuracy = (matriz_confusion[0, 0] + matriz_confusion[1, 1]) / (matriz_confusion.sum())

# Imprime las métricas
print(f"Precisión: {precision}")
print(f"Especificidad: {specificity}")
print(f"Recall: {recall}")
print(f"Fallout: {fallout}")
print(f"Accuracy: {accuracy}")

[[191.   3.]
 [  8.  63.]]
Precisión: 0.9597989949748744
Especificidad: 0.9545454545454546
Recall: 0.9845360824742269
Fallout: 0.11267605633802817
Accuracy: 0.9584905660377359
